In [6]:
import numpy as np
import cv2 as cv

def create_gradient_map(c1, c2):
    b = np.linspace(c1[0], c2[0], 256)
    g = np.linspace(c1[1], c2[1], 256)
    r = np.linspace(c1[2], c2[2], 256)   
    grad_map = [(int(b[i]), int(g[i]), int(r[i])) for i in range(256)]
    return grad_map

def show_grad_map(gmap):
    cv.namedWindow('gradient map', cv.WINDOW_NORMAL)
    cv.resizeWindow('gradient map', 500, 500)
    l = len(gmap)
    to_show = np.zeros((2*l, 2*l, 3), dtype=np.uint8)
    for i, (b,g,r) in enumerate(gmap):
        cv.line(to_show, (0, 2*i), (2*l, 2*i), (int(b),int(g),int(r)), 2)
    cv.imshow('gradient map', to_show)

def create_grad_img(img, grad_map):
    _,_,img_v = cv.split(cv.cvtColor(img, cv.COLOR_BGR2HSV))
    grad_img = np.zeros_like(img)
    for i, (b,g,r) in enumerate(grad_map):
        grad_img[img_v==i] = [b,g,r]
    return grad_img

def blend_images(img1, img2, alpha=0.5):
    return cv.addWeighted(img2, alpha, img1, 1-alpha, 0)

In [17]:
#create a video that swipes images from blend=1 to blend=0
PRE_TIME = 0.0 #seconds
TIME = 5.0 #seconds
POST_TIME = 1.0 #seconds
FPS = 60.0 #frames per second

BLEND_START = 0.8
BLEND_END = 0.0

B_COL = (255, 200, 0)  #bgr
W_COL = (255, 0, 255)  #bgr
img = cv.imread('img.jpg')

grad_map = create_gradient_map(B_COL, W_COL)
grad_img = create_grad_img(img, grad_map)

cv.namedWindow('video', cv.WINDOW_NORMAL)
cv.resizeWindow('video', 800, 800)

show_grad_map(grad_map)


#video
assert (PRE_TIME+TIME+POST_TIME)*FPS > len(grad_map), f'too short, TIM*FPS: {TIME*FPS}, len(grad_map): {len(grad_map)}'
tot_frames_single_color = int(int(TIME*FPS)/len(grad_map))
tot_frames = tot_frames_single_color*len(grad_map)
blends = np.linspace(BLEND_START, BLEND_END, tot_frames)
print(f'tot_frames: {tot_frames}')
#create video with maximum quality
video = cv.VideoWriter('demo.mp4', cv.VideoWriter_fourcc(*'XVID'), FPS, (img.shape[1], img.shape[0]))
#add pre frames
for i in range(int(PRE_TIME*FPS)):
    blend_image = blend_images(img, grad_img, blends[0])
    video.write(blend_image)
    cv.imshow('video', blend_image)
    cv.waitKey(10)
for i in range(tot_frames):
    blend_image = blend_images(img, grad_img, blends[i])
    #add frame to video
    video.write(blend_image)
    cv.imshow('video', blend_image)
    cv.waitKey(10)
#add post frames
for i in range(int(POST_TIME*FPS)):
    blend_image = blend_images(img, grad_img, blends[-1])
    video.write(blend_image)
    cv.imshow('video', blend_image)
    cv.waitKey(10)
video.release()

cv.destroyAllWindows()

tot_frames: 256


OpenCV: FFMPEG: tag 0x44495658/'XVID' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
